In [3]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


def scrape(keyword):
    # Configure Chrome WebDriver
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Run in headless mode (without opening the browser window)
    driver = webdriver.Chrome(options=options)

    # Navigate to Google Maps
    driver.get(keyword)

    # Wait for the search results to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#hdtb-msb > div:nth-child(1) > div > div:nth-child(2) > a')))

    # Scrape data from multiple pages (2 to 3 in this example)
    data = []
    for p in range(2, 4):
        cid_elements = driver.find_elements(By.CSS_SELECTOR, 'div#search a[data-cid]')
        for element in cid_elements:
            cid = element.get_attribute("data-cid")
            business_name = element.find_element(By.CSS_SELECTOR, 'div[role="heading"]').text

            # Scrape details for each business
            cid_element = driver.find_element(By.CSS_SELECTOR, f'div#search a[data-cid="{cid}"]')
            cid_element.click()

            # Wait for the details to load
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h2[data-attrid="title"]')))

            business_data = {
                "Name": business_name,
                "Address": "",
                "Phone": "",
                "Rating": "",
                "ReviewsCount": "",
                "ImageURL": "",
                "Category": "",
                "Reviews": []
            }

            business_data["Address"] = driver.find_element(By.CSS_SELECTOR, 'div[data-attrid="kc:/location/location:address"] span:nth-child(2)').text
            business_data["Phone"] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label*="Call"]').text
            business_data["Rating"] = driver.find_element(By.CSS_SELECTOR, 'g-review-stars span').get_attribute('aria-label')
            business_data["ReviewsCount"] = driver.find_element(By.CSS_SELECTOR, 'a[data-async-trigger="reviewDialog"] span').text
            business_data["ImageURL"] = driver.find_element(By.CSS_SELECTOR, 'img[alt="Merchant logo"]').get_attribute('src')
            business_data["Category"] = driver.find_element(By.CSS_SELECTOR, 'div[data-attrid="kc:/local:lu attribute list"] > div > div > span').text

            # Check if reviews are available
            if driver.find_element(By.CSS_SELECTOR, 'a[data-async-trigger="reviewDialog"]') is not None:
                reviews_btn = driver.find_element(By.CSS_SELECTOR, 'a[data-async-trigger="reviewDialog"]')
                reviews_btn.click()
                time.sleep(2)

                # Click on "More" buttons to load all reviews
                while True:
                    try:
                        more_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.review-more-link')
                        if len(more_buttons) == 0:
                            break
                        for btn in more_buttons:
                            btn.click()
                            time.sleep(1)
                    except:
                        break

                # Scrape reviews
                review_author_names = driver.find_elements(By.CSS_SELECTOR, 'div[data-google-review-count] > div > div > div > div > a')
                review_author_images = driver.find_elements(By.CSS_SELECTOR, 'div[data-google-review-count] > div > a > img')
                review_ratings = driver.find_elements(By.CSS_SELECTOR, 'div[data-google-review-count] div g-review-stars > span')
                review_dates = driver.find_elements(By.CSS_SELECTOR, 'div[data-google-review-count] div g-review-stars + span')
                review_texts = driver.find_elements(By.CSS_SELECTOR, 'div[data-google-review-count] div > div > div > div > span[jscontroller]')

                for j in range(len(review_author_names)):
                    review_author_name = review_author_names[j].text
                    review_author_image = review_author_images[j].get_attribute('src')
                    review_rating = review_ratings[j].get_attribute('aria-label')
                    review_date = review_dates[j].text
                    review_text = review_texts[j].text.replace("'", "")  # Remove single quotes from review text

                    review_data = {
                        "AuthorName": review_author_name,
                        "AuthorImage": review_author_image,
                        "Rating": review_rating,
                        "Date": review_date,
                        "Text": review_text
                    }

                    business_data["Reviews"].append(review_data)

                # Go back to the business details page
                driver.back()
                time.sleep(2)

            data.append(business_data)

        # Go to the next page
        page_button = driver.find_element(By.CSS_SELECTOR, f'a[aria-label="Page {p}"]')
        page_button.click()
        time.sleep(2)

    # Close the browser
    driver.quit()

    # Export data as JSON
    with open('scraped_data.json', 'w') as file:
        json.dump(data, file, indent=4)

    # Export data to SQL
    for business in data:
        sql = "INSERT INTO ChandigarhHarals (Name, Address, Phone, Rating, ReviewsCount, ImageURL, Category) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (
            business["Name"], business["Address"], business["Phone"], business["Rating"],
            business["ReviewsCount"], business["ImageURL"], business["Category"]
        ))
        connection.commit()
        print("1 record inserted")

        for review in business["Reviews"]:
            sql = "INSERT INTO ChandigarhHaralsReviews (Name, Photo, ReviewText, Rating, ReviewedDate) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(sql, (
                review["AuthorName"], review["AuthorImage"], review["Text"], review["Rating"], review["Date"]
            ))
            connection.commit()
            print("1 review inserted")

# Define the Google Maps URL
keyword = 'https://www.google.com/search?q=' + 'dentists+in+kathmandu' + '&hl=en'

# Call the scrape function
scrape(keyword)

# Close the database connection
cursor.close()
connection.close()


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001004ff8ac chromedriver + 4257964
1   chromedriver                        0x00000001004f7f40 chromedriver + 4226880
2   chromedriver                        0x00000001001349d4 chromedriver + 281044
3   chromedriver                        0x000000010016fa34 chromedriver + 522804
4   chromedriver                        0x00000001001a67e4 chromedriver + 747492
5   chromedriver                        0x000000010016398c chromedriver + 473484
6   chromedriver                        0x000000010016498c chromedriver + 477580
7   chromedriver                        0x00000001004be900 chromedriver + 3991808
8   chromedriver                        0x00000001004c2354 chromedriver + 4006740
9   chromedriver                        0x00000001004c2940 chromedriver + 4008256
10  chromedriver                        0x00000001004c833c chromedriver + 4031292
11  chromedriver                        0x00000001004c2f34 chro